In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import time
import pickle
from dateutil import relativedelta
import seaborn as sns

### API call for redcap

In [ ]:
def api_call(url, query, logger=None):
    """ helper function to make API calls to RedCap
    """
    r = requests.post(url, data=query, verify=False)
    http_status = str(r.status_code)
    print(f'HTTP Status: {http_status}')

    if http_status == "200":
        query_results = r.json()
        query_df = pd.DataFrame(query_results)

    else:
        print(f"RedCap API request Failed with HTTP Status: {http_status}")
        query_df = None
        
    return query_df

def get_inventory_count(df, index_col, availability_indicators):
    """ helper function to count participants with recorded data in redcap
    """
    assess_cols = df.columns.drop(index_col)

    if availability_indicators == 'number':
        df = df.replace("", np.nan)
        df[assess_cols] = df[assess_cols].astype(np.float64)

    inventory = {}
    for col in assess_cols:        
        if availability_indicators == 'number':
            availability_count = df[~df[col].isna()][index_col].nunique()
        else:
            availability_count = df[df[col].isin(availability_indicators)][index_col].nunique()
        inventory[col] = availability_count
    return inventory

def get_available_data(config_json, DATASET_ROOT, var_name, preferred_var_source="primary"):
    """ Get data for given variables from available sources
        All return dataframes should have participant_id and visit_id as index
    """
    config_data = json.load(open(config_json))
    data_sources = config_data['data_sources']
    variable_info = config_data['variables'][var_name]
    variable_type = variable_info["type"]
    variable_sources = variable_info["sources"]

    if preferred_var_source == "primary":
        selected_var_source = variable_info['primary_source']
        selected_var_instrument = variable_info['primary_instrument']
    elif preferred_var_source == "secondary":
        selected_var_source = variable_info['secondary_source']
        selected_var_instrument = variable_info['secondary_instrument']
    else:
        print(f"Using preferred source {preferred_var_source} for variable {var_name}")
        preferred_var_data_source = preferred_var_source["data_source"]
        preferred_var_instrument = preferred_var_source["instrument"]

        if preferred_var_data_source not in variable_sources.keys():
            print(f"Preferred data source {preferred_var_data_source} not available for variable {var_name}")
            return None
        else:
            selected_var_source = preferred_var_data_source

        if preferred_var_instrument not in variable_sources[selected_var_source].keys():
            print(f"Preferred var instrument {preferred_var_instrument} not available for variable {var_name}")
            return None
        else:
            selected_var_instrument = preferred_var_instrument

    print(f"Using variable {var_name} from source {selected_var_source} and instrument {selected_var_instrument}")

    external_var_cols = variable_sources[selected_var_source][selected_var_instrument]

    # Get data from primary source
    var_file = data_sources[selected_var_source][selected_var_instrument]["path"]
    var_file_path = f"{DATASET_ROOT}/{var_file}"
    var_file_index = data_sources[selected_var_source][selected_var_instrument]["index_cols"]

    var_df = pd.read_csv(var_file_path)
    selected_var_cols = list(set(var_file_index + external_var_cols))
    var_df = var_df[selected_var_cols]
    
    if (variable_type == "date") & (len(external_var_cols) == 1):
        var_df[external_var_cols[0]] = pd.to_datetime(var_df[external_var_cols[0]], errors="coerce", dayfirst=False)

    if (len(external_var_cols) == 1):
        var_df = var_df.rename(columns={external_var_cols[0]:var_name})
    
    return var_df


### Paths


In [ ]:
DATASET_ROOT = "/home/nikhil/projects/Parkinsons/qpn/"

# Current nipoppy manifest
release_dir = f"{DATASET_ROOT}/releases/"
current_release = "Jan_2024"

tabular_data_release_dir = f"{release_dir}/{current_release}/"

demo_config_json = "../workflow/tabular/demographics.json"
pheno_config_json = "../workflow/tabular/pheno.json"


### Standardized index names

In [ ]:
baseline_event_name = "Baseline (Arm 1: C-OPN)"

## redcap event name variations
config_data = json.load(open(demo_config_json))
data_sources = config_data['data_sources']
redcap_data_sources = data_sources['redcap']

redcap_field_name_map = {}

for instrument in redcap_data_sources.keys():
    index_cols = redcap_data_sources[instrument]['index_cols']
    record_id = index_cols[0]
    event_name = index_cols[1]

    redcap_field_name_map[record_id] = "participant_id"
    redcap_field_name_map[event_name] = "redcap_event_name"
print(f"redcap_field_name_map: {redcap_field_name_map}")

# legacy participant_id variations in DOB and BD_RPQ
legacy_field_name_map = {}
legacy_field_name_map['Record ID'] = "participant_id"
legacy_field_name_map['Patient #'] = "participant_id"
legacy_field_name_map['Name of visit (V01, V02, V03)'] = "visit"
print(f"legacy_field_name_map: {legacy_field_name_map}")

### Update RedCAP reports through API 
(Not updating extended report since it has to come from Sarah)
- "global_records_query"
- "QPN MoCA-UPDRS-Neuropsy data_Sarah"

In [ ]:
update_redcap_reports = False

redcap_report_list = ["global_records_query", "QPN MoCA-UPDRS-Neuropsy data_Sarah"]
if update_redcap_reports:
    redcap_config_json = f"{DATASET_ROOT}/proc/.redcap.json"
    redcap_config = json.load(open(redcap_config_json))
    url = redcap_config["url"]
    
    for redcap_report in redcap_report_list:
        print(f"Getting data for RedCap report: {redcap_report}")
        records_query = redcap_config["queries"][redcap_report]
        query_df = api_call(url, records_query, logger=None)
        report_csv = f"{release_dir}{current_release}/tabular/redcap/{redcap_report}.csv"
        query_df.to_csv(report_csv, index=False)
        print(f"Saved RedCap report to {report_csv}")



### Available participants

In [ ]:
QPN_participants_df = get_available_data(demo_config_json,tabular_data_release_dir,"participant_id")
QPN_participants = QPN_participants_df["participant_id"].unique()
n_participants = len(QPN_participants)
print(f"Number of participants: {n_participants}")

### Fetch demographic data

In [ ]:
_df2

In [ ]:
demo_vars = ["dob", "group", "sex"]
# preferred_var_source = {"data_source":"local","instrument":"legacy_DOB"}
vars_with_secondary_source = ["dob"]

config_json = demo_config_json
index_cols = ["participant_id", "redcap_event_name"]

demo_var_df = pd.DataFrame()
for var in demo_vars:
    _df = get_available_data(config_json,tabular_data_release_dir,var)
    _df = _df.rename(columns=redcap_field_name_map)
    _df = _df.rename(columns=legacy_field_name_map)
    _df = _df[_df["participant_id"].isin(QPN_participants)].copy()

    if var in vars_with_secondary_source:
        print(f"**Getting data from the secondary source for variable {var}**")
        _df2 = get_available_data(config_json,tabular_data_release_dir,var,preferred_var_source="secondary")
        _df2 = _df2.rename(columns=legacy_field_name_map)
        _df2 = _df2.rename(columns={var:var+"_secondary"})
        _df2 = _df2[_df2["participant_id"].isin(QPN_participants)].copy()
        
        # Merge primary and secondary sources
        n_missing_in_primary = _df[_df["redcap_event_name"]==baseline_event_name][var].isna().sum()
        print(f"Missing data in primary source: {n_missing_in_primary}")

        if "redcap_event_name" in _df2.columns:
            _df = pd.merge(_df, _df2, on=["participant_id","redcap_event_name"], how="outer")
        else:
            _df = pd.merge(_df, _df2, on="participant_id", how="outer")
        _df[var] = _df[var].fillna(_df[var+"_secondary"])
        # _df = _df.drop(columns=[var+"_secondary"])

        n_missing_after_secondary_fill = _df[_df["redcap_event_name"]==baseline_event_name][var].isna().sum()
        print(f"Missing data after secondary source fill: {n_missing_after_secondary_fill}")

    if demo_var_df.empty:
        demo_var_df = _df
    else:
        demo_var_df = pd.merge(demo_var_df, _df, on=index_cols, how="outer")   

demo_participants = demo_var_df["participant_id"].unique()
n_demo_participants = len(demo_participants)
print('-'*50)
print(f"Number of participants with demographics data: {n_demo_participants}")
print('-'*50)

demo_redcap_events = demo_var_df["redcap_event_name"].unique()
print(f"Demographics data available for events: {demo_redcap_events}")
print('-'*50)

for var in demo_vars:
    n_unique = demo_var_df[demo_var_df["redcap_event_name"]=="Baseline (Arm 1: C-OPN)"][var].nunique()
    n_missing = demo_var_df[demo_var_df["redcap_event_name"]=="Baseline (Arm 1: C-OPN)"][var].isna().sum()
    print(f"Var: {var}, n_unique: {n_unique}, n_missing: {n_missing} (out of {n_demo_participants})")

demo_var_df.head()

### Find records with phenotypic data

In [ ]:
pheno_vars = ["diagnosis", "updrs_scores", "moca_scores", "diagnosis_date", "updrs_date", "moca_date"]
# preferred_var_source = {"data_source":"local","instrument":"legacy_DOB"}

config_json = pheno_config_json
index_cols = ["participant_id", "redcap_event_name"]
pheno_var_df = pd.DataFrame()
for var in pheno_vars:
    _df = get_available_data(config_json,tabular_data_release_dir,var)
    _df = _df.rename(columns=redcap_field_name_map)
    _df = _df.rename(columns=legacy_field_name_map)
    _df = _df[_df["participant_id"].isin(QPN_participants)].copy()
    if pheno_var_df.empty:
        pheno_var_df = _df
    else:
        pheno_var_df = pd.merge(pheno_var_df, _df, on=index_cols, how="outer")   

pheno_participants = pheno_var_df["participant_id"].unique()
n_pheno_participants = len(pheno_participants)
print('-'*50)
print(f"Number of participants with pheno data: {n_pheno_participants}")
print('-'*50)

pheno_redcap_events = pheno_var_df["redcap_event_name"].unique()
print(f"Pheno data available for events: {pheno_redcap_events}")
print('-'*50)

for var in pheno_var_df.columns:
    for redcap_event in pheno_redcap_events:
        if var not in index_cols:
            pheno_var_event_df = pheno_var_df[pheno_var_df["redcap_event_name"]==redcap_event].copy()
            n_pheno_var_event_participants = pheno_var_event_df["participant_id"].nunique()
            if pheno_var_event_df[var].nunique() > 0:    
                print(f"Var: {var}, Event: {redcap_event}")
                n_unique = pheno_var_event_df[var].nunique()
                n_missing = pheno_var_event_df[var].isna().sum()
                print(f"n_unique: {n_unique}, n_missing: {n_missing} (out of {n_pheno_var_event_participants})")
    print('-'*50)

pheno_var_df.head()

### Add mri_acq date
- Needs to map to redcap_event_name

In [ ]:
var = "MRI_date"
config_json = pheno_config_json
mri_date_df = get_available_data(config_json,tabular_data_release_dir,var)
mri_date_df["MRI_date"] = pd.to_datetime(mri_date_df["MRI_date"], errors="coerce", dayfirst=False)

n_mri_participants = mri_date_df["participant_id"].nunique()
print(f"Number of participants with MRI data: {n_mri_participants}")

n_sessions = mri_date_df["session"].nunique()
print(f"Number of MRI sessions: {n_sessions}")

participants_with_follow_ups = mri_date_df[mri_date_df["participant_id"].duplicated()]["participant_id"].unique()
n_participants_with_follow_ups = len(participants_with_follow_ups)
print(f"Number of participants with follow-up MRI: {n_participants_with_follow_ups}")

mri_ses01_date_df = mri_date_df[mri_date_df["session"]=="ses-01"].copy()
mri_ses01_date_df["redcap_event_name"] = "Baseline (Arm 1: C-OPN)"

mri_ses02_date_df = mri_date_df[mri_date_df["session"]=="ses-02"].copy()
mri_ses02_participants = mri_ses02_date_df["participant_id"].unique()
print(f"Number of participants with ses-02 MRI: {len(mri_ses02_participants)}")

baseline_df = mri_ses01_date_df[mri_ses01_date_df["participant_id"].isin(mri_ses02_participants)].set_index("participant_id")
followup_df = mri_ses02_date_df.set_index("participant_id")

visit_months = [12, 18, 24, 30, 36, 42, 48, 54]
month_bins = [9, 15, 21, 27, 33, 39, 45, 51, 57]

event_str_suffix = "Months Follow-Up/Suivi (Arm 1: C-OPN)"
event_names = [f"{m} {event_str_suffix}" for m in visit_months]

# --- Bin the months --- #
followup_df["months_since_baseline"] = followup_df["MRI_date"].dt.to_period('M').astype(int) - baseline_df["MRI_date"].dt.to_period('M').astype(int)
followup_df["months_since_baseline"] = followup_df["months_since_baseline"].replace({0:np.nan}) # Some visits get same acq_date from brodacasting merge. 

followup_df["redcap_event_name"] = pd.cut(followup_df["months_since_baseline"], bins=month_bins, labels=event_names)

mri_date_redcap_event_df = pd.concat([mri_ses01_date_df, followup_df.reset_index()], axis=0)
# mri_date_redcap_event_df = mri_date_redcap_event_df

mri_date_redcap_event_df.sort_values(["participant_id","session"]).head()

#### Add MRI date to pheno data

In [ ]:
pheno_var_df = pd.merge(pheno_var_df, mri_date_redcap_event_df, on=index_cols, how="right")  
var = "MRI_date"
for redcap_event in mri_date_redcap_event_df["redcap_event_name"].unique():    
    pheno_var_event_df = pheno_var_df[pheno_var_df["redcap_event_name"]==redcap_event].copy()
    n_pheno_var_event_participants = pheno_var_event_df["participant_id"].nunique()
    if pheno_var_event_df[var].nunique() > 0:    
        print(f"Var: {var}, Event: {redcap_event}")
        n_unique = pheno_var_event_df[var].nunique()
        n_missing = pheno_var_event_df[var].isna().sum()
        print(f"n_unique: {n_unique}, n_missing: {n_missing} (out of {n_pheno_var_event_participants})")
pheno_var_df.head()

### Neuropsych data
- Comes from either from Sarah's extended report or BD_RPQ_UPDATE_Neuropsy

In [ ]:
neuropsych_vars = ["neuropsy_scores","neuropsy_date"]

index_cols = ["participant_id", "visit", "TimePoint (based on REDCap; baseline, 18m, 36m, etc.)", "Délai depuis baseline (mois)"]
neuropsych_df = pd.DataFrame()
for var in neuropsych_vars:
    _df = get_available_data(config_json,tabular_data_release_dir,var)
    _df = _df.rename(columns=redcap_field_name_map)
    _df = _df.rename(columns=legacy_field_name_map)
    _df = _df[_df["participant_id"].isin(QPN_participants)].copy()
    if neuropsych_df.empty:
        neuropsych_df = _df
    else:
        neuropsych_df = pd.merge(neuropsych_df, _df, on=index_cols, how="left")   

neuropsych_participants = neuropsych_df["participant_id"].unique()
n_neuropsych_participants = len(neuropsych_participants)
print('-'*50)
print(f"Number of participants with neuropysch data: {n_neuropsych_participants}")
print('-'*50)

neuropsych_visits = neuropsych_df["visit"].unique()
print(f"neuropsych data available for events: {neuropsych_visits}")
print('-'*50)

neuropsych_df.head()

### Basic clean-up and data checks

In [ ]:
# Fix dtypes
for series_name, series in neuropsych_df.items():
    if "score" in series_name:
        if series.dtype == 'object':
            print(f"recasting {series_name} to float by replacing , with .")
            neuropsych_df[series_name] = neuropsych_df[series_name].str.replace(",",".").astype(float)
            neuropsych_df.loc[neuropsych_df[series_name]>900, series_name] = np.nan
            
    # Replace >900 with NaNs
    if series.dtype == 'float':
        neuropsych_df.loc[neuropsych_df[series_name]>900, series_name] = np.nan

# assign redcap_event_name
visit_months = [12, 18, 24, 30, 36, 42, 48, 54]
month_bins = [9, 15, 21, 27, 33, 39, 45, 51, 57]
event_str_suffix = "Months Follow-Up/Suivi (Arm 1: C-OPN)"
event_names = [f"{m} {event_str_suffix}" for m in visit_months]

neuropsych_df["redcap_event_name"] = pd.cut(neuropsych_df["Délai depuis baseline (mois)"], bins=month_bins, labels=event_names).astype(str)
neuropsych_df.loc[neuropsych_df["TimePoint (based on REDCap; baseline, 18m, 36m, etc.)"]=="baseline", 
                  "redcap_event_name"] = "Baseline (Arm 1: C-OPN)"

# Merge with pheno_var_df
index_cols = ["participant_id", "redcap_event_name"]
pheno_var_df = pd.merge(pheno_var_df, neuropsych_df, on=index_cols, how="left")  

pheno_var_df.head()

### Calculate age

In [ ]:
demo_cols = ["participant_id", "dob", "group", "sex"]
demo_var_df[demo_var_df["participant_id"]==participants_with_follow_ups[0]]
baseline_demo_df = demo_var_df[demo_var_df["redcap_event_name"]=="Baseline (Arm 1: C-OPN)"][demo_cols].copy()

index_cols = ["participant_id"] # not using redcap_event_name to allow broadcast of demographics vars
tabular_df = pd.merge(pheno_var_df, baseline_demo_df, on=index_cols, how="left")
tabular_df[tabular_df["participant_id"]==participants_with_follow_ups[0]]

tabular_df.head()

In [ ]:
def get_age_at_visit(df, var, dob_col="dob", rounding_digits=2, age_range=(0,100)):
    """ Get age at visit. Expects column name to be: var_date """
    
    age_col = var.split("_")[0]+"_age"
    df[age_col] = df[var] - tabular_df[dob_col]
    df[age_col] = np.round(df[age_col].dt.days / 365.25, rounding_digits)

    if (len(df[df[age_col] > 100]) | len(df[df[age_col] < 0])):
        print(f"Warning: Age values outside range {age_range} for variable {var}")

    return df



In [ ]:
age_vars = ["diagnosis_date", "updrs_date", "moca_date", "MRI_date", "neuropsy_date"]

for age_var in age_vars:
    tabular_df = get_age_at_visit(tabular_df, age_var)

tabular_df.head()

### QPN paper tables

#### Demo table

In [ ]:
demo_vars = ["participant_id", "redcap_event_name", "MRI_age", "sex", "group"]
redcap_events = ["Baseline (Arm 1: C-OPN)","12 Months Follow-Up/Suivi (Arm 1: C-OPN)","18 Months Follow-Up/Suivi (Arm 1: C-OPN)"]
QPN_groups = {"Healthy control/Contrôle": "control", 
              "PD   (Parkinson's Disease)/Maladie de Parkinson": "PD"}
QPN_sexes = {"Female/Féminin": "Female", "Male/Masculin":"Male"}
n_tabular_participants = tabular_df["participant_id"].nunique()
print(f"Number of participants: {n_tabular_participants}")

demo_df = tabular_df[(tabular_df["redcap_event_name"].isin(redcap_events)) & 
                     (tabular_df["group"].isin(QPN_groups.keys()))][demo_vars].copy()

demo_df["group"] = demo_df["group"].replace(QPN_groups)
demo_df["sex"] = demo_df["sex"].replace(QPN_sexes)

n_participants = demo_df["participant_id"].nunique()
print(f"Number of participants after event and group filter: {n_participants}")

demo_df.head()

In [ ]:
# counts
global_count_df = demo_df.groupby(["redcap_event_name"])["MRI_age"].count().reset_index()
global_count_df = global_count_df.rename(columns={"MRI_age":"n_participants"})

# age
mean_age_df = demo_df.groupby(["redcap_event_name"])["MRI_age"].mean().round(1).reset_index()
std_age_df = demo_df.groupby(["redcap_event_name"])["MRI_age"].std().round(1).reset_index()

demo_table_df = pd.merge(global_count_df, mean_age_df, on="redcap_event_name")
demo_table_df = pd.merge(demo_table_df, std_age_df, on="redcap_event_name", suffixes=('_mean', '_std'))

# sex
sex_count_df = demo_df.groupby(["redcap_event_name"])["sex"].value_counts().unstack().reset_index()
demo_table_df = pd.merge(demo_table_df, sex_count_df, on="redcap_event_name")

# group
group_count_df = demo_df.groupby(["redcap_event_name"])["group"].value_counts().unstack().reset_index()
demo_table_df = pd.merge(demo_table_df, group_count_df, on="redcap_event_name")

demo_table_df.head()


#### Pheno table